#  Étude de Cas : Planification de pièces à usiner sur des machines de production
 Dans ce notebook, nous allons résoudre un problème d'optimisation de l'assignation de pièces à usiner sur des machines de production dans une usine en utilisant **la programmation linéaire (PL)** et une **heuristique gloutonne (LPT - Longest Processing Time First)**.
 ##  Objectif
 - Minimiser le **temps total d'usiange** des pieces (makespan).
 - Comparer la **PL** avec une **heuristique simple**.

##  1. Définition du Problème
 Nous avons un ensemble de **Pièces** qui doivent être usinées sur un ensemble de **Machines**.
 - Chaque **pièce a un temps d'exécution**.
 - Chaque **machine a une vitesse de traitement**.
 - Une pièce ne peut être exécutée que sur **une seule machine**.
 - L'objectif est de **minimiser le temps total d'exécution**.

 ### 1.1. Modèle PL
 Ci-dessous, completez le modèle PL pour ce problème.

 - **Variables de décision** :
   - $x_{ij} = 1$ si la pièce $i$ est assignée à la machine $j$, sinon $0$.
   - $C_{max}$ est le makespan (temps d'exécution total à minimiser).

 - **Fonction objectif** :
    > A COMPLETER

 - **Contraintes** :
   1. Chaque pièce est usinée exactement **une seule fois** :
    > A COMPLETER
   
   2. Le makespan doit être supérieur ou égal au **temps d'usinage total** sur chaque machine (Aide: Le temps d'exécution total sur la machine $j$ est égal à la somme des temps d'exécution des tâches assignées à $j$ divisée par la **vitesse de traitement** de $j$) :
    > A COMPLETER
   
   Où $T_i$ est la durée de la pièce $i$ et $S_j$ la vitesse de la machine $j$.
   ### 1.2. Implémentation

   Complétez le code suivant:

In [ ]:
!pip install pulp
import numpy as np
import pulp

# Définition des tâches et machines
pieces = [10, 20, 30, 40, 50]  # Temps d'usinage des pièces
machines = [1.0, 1.5, 2.0]  # Puissances des machines (usinage plus rapide si valeur plus grande)

def LP(pieces,machines):
  num_pieces = len(pieces)
  num_machines = len(machines)

  # Création du modèle PL
  model = pulp.LpProblem("Cluster_Scheduling", """A COMPLETER""")

  # Variables de décision
  x = pulp.LpVariable.dicts("x", [(i, j) for i in range("""A COMPLETER""") for j in range("""A COMPLETER""")], cat="""A COMPLETER""")
  Cmax = pulp.LpVariable("Cmax", lowBound=0, cat="""A COMPLETER""")  # Temps d'usinage max

  # Fonction Objectif : Minimiser le makespan
  model += Cmax

  # Contrainte : Chaque pièces doit être affectée à une seule machine
  for i in range(num_pieces):
      model += """A COMPLETER"""

  # Contrainte : Calculer le makespan (temps max sur toutes les machines)
  for j in range(num_machines):
      model += pulp.lpSum("""A COMPLETER""" * (pieces[i] / machines[j]) for i in range("""A COMPLETER""")) <= """A COMPLETER"""

  # Résolution
  model.solve()
  return Cmax,x
# Résultats
Cmax,x = LP(pieces,machines)
print("\n🔹 Solution Optimale (PL) :")
print(f"Makespan Min : {pulp.value(Cmax)}")
for i in range(len(pieces)):
    for j in range(len(machines)):
        if pulp.value(x[(i, j)]) == 1:
            print(f"Pièce {i+1} assignée à Machine {j+1}")

##  2. Heuristique Gloutonne : Longest Processing Time (LPT)
 LPT est une méthode simple qui **affecte d'abord les pièces les plus longues à usiner aux machines les moins chargées**.

 **Comment fonctionne LPT ?**
 1. Trier les pièces **par ordre décroissant** de durée d'usinage.
 2. Affecter chaque pièces à **la machine la moins chargée**.
 3. Mettre à jour la charge de la machine et continuer.

 Le code de LPT vous est fourni ci dessous, ainsi qu'un exemple d'execution sur la même instance que le programme linéaire précédent.

In [ ]:
# Implémentation de l'heuristique LPT
def LPT(pieces,machines):
  pieces_order = sorted(enumerate(pieces), key=lambda x: x[1], reverse=True)  # Trier par durée décroissante
  machine_loads = [0] * len(machines)  # Charge actuelle des machines
  assignment = {}  # Stocke l'affectation des pièces

  for i, duration in pieces_order:
      best_machine = np.argmin(machine_loads)  # Trouver la machine la moins chargée
      machine_loads[best_machine] += duration / machines[best_machine]
      assignment[i] = best_machine
  return machine_loads,assignment

machine_loads, assignment = LPT(pieces,machines)

# Résultats de l'heuristique
print("\n🔹 Solution Heuristique (LPT) :")
print(f"Makespan Estimé : {max(machine_loads)}")
for i in range(len(pieces)):
    print(f"Pièce {i+1} assignée à Machine {assignment[i]+1}")

##  3. Évaluation des performances
 Nous voulons comparer les différentes approches en terme de **temps d'usinage** et de **qualité de la solution**.

 Complétez le code ci dessous qui compare les deux approches sur l'instance donnée.


In [ ]:
import time

# Définition des pièces et machines
pieces = [10, 20, 30, 40, 50]  # Temps d'usinage des pièces
machines = [1.0, 1.5, 2.0]  # Puissances des machines (usinage plus rapide si valeur plus grande)

# Mesurer le temps d'usinage du PL
start_time = time.time()
Cmax,_ = """A COMPLETER"""
pl_time = time.time() - start_time

# Mesurer le temps d'usinage de LPT
start_time = time.time()
machine_loads,_ = """A COMPLETER"""
lpt_time = """A COMPLETER"""

# Comparaison qualité de solution
pl_makespan = pulp.value(Cmax)
lpt_makespan = max(machine_loads)

print("\n🔹 Évaluation des performances :")
print(f"Temps d'usinage PL : {pl_time:.4f} sec")
print(f"Temps d'usinage LPT : {lpt_time:.4f} sec")
print(f"Makespan PL : {pl_makespan}")
print(f"Makespan LPT : {lpt_makespan}")
print(f"Qualité de solution LPT vs PL : {100 * lpt_makespan / pl_makespan:.2f}%")

Il parait évident que pour cette petite instance, le programme linéaire donne une solution bien meilleure que LPT. Mais est-ce toujours le cas ?
 Complétez la cellule ci dessous pour tester sur un grand nombre d'instances:

In [ ]:
import time
import random
nb_instances = 100

exec_time_LP = 0
exec_time_LPT = 0
value_LP = 0
value_LPT = 0

for _ in range("""A COMPLETER"""):

  # Définition des pièces et machines
  pieces = [random.randint(1,8)*10 for _ in range(8)]
  machines = [random.uniform(1, 3) for _ in range(3)]  # Puissances des machines (usinage plus rapide si valeur plus grande)

  # Mesurer le temps d'exécution du PL
  start_time = time.time()
  Cmax,_ = """A COMPLETER"""
  value_LP += pulp.value(Cmax)
  exec_time_LP += time.time() - start_time
  # Mesurer le temps d'exécution de LPT
  start_time = time.time()
  machine_loads,_ = """A COMPLETER"""
  value_LPT += max(machine_loads)
  exec_time_LPT += time.time() - start_time


print("\n🔹 Évaluation des performances :")
print(f"Temps d'usinage moyen PL : {exec_time_LP/nb_instances:.4f} sec")
print(f"Temps d'usinage moyen LPT : {exec_time_LPT/nb_instances:.4f} sec")
print(f"Makespan moyen PL : {value_LP/nb_instances}")
print(f"Makespan moyen LPT : {value_LPT/nb_instances}")
print(f"Qualité de solution LPT vs PL : {100 * value_LPT / value_LP:.2f}%")

**Quel algorithme semble meilleur en terme de valeur de solution?**

> A COMPLETER

 **Et de temps de calcul ?**
> A COMPLETER

 **Pouvez vous donner le nombre de pieces a partir de laquelle il n'est plus envisageable d'utiliser LP (i.e. une instance met plus de 1 seconde à calculer)?**      
> A COMPLETER

**Dans ce cas (où nous avons de grandes instances), comment pouvons nous utiliser la programation linéaire pour avoir une indication sur la qualité de la solution ? Complétez le code ci dessous en conséquence**
 > A COMPLETER

In [8]:
def Mystery_fct(pieces,machines):
  num_pieces = len(pieces)
  num_machines = len(machines)

  # Création du modèle PL
  model = pulp.LpProblem("Cluster_Scheduling", pulp.LpMinimize)

  # Variables de décision
  x = """A COMPLETER"""
  Cmax = pulp.LpVariable("Cmax", lowBound=0, cat=pulp.LpContinuous)  # Temps d'exécution max

  # Fonction Objectif : Minimiser le makespan
  model += Cmax

  # Contrainte : Chaque pièces doit être affectée à une seule machine
  for i in range(num_pieces):
      model += pulp.lpSum(x[(i, j)] for j in range(num_machines)) == 1

  # Contrainte : Calculer le makespan (temps max sur toutes les machines)
  for j in range(num_machines):
      model += pulp.lpSum(x[(i, j)] * (pieces[i] / machines[j]) for i in range(num_pieces)) <= Cmax
  # Résolution
  model.solve()
  return Cmax,x


 Completez alors le code ci dessous, qui trace la valeur moyenne de la solution de LPT comparé au résultat de la fonction précédente pour un nombre de tâches croissants. Vous pouvez bien entendu changer le nom des variables "mystery" et les labels "mystère" par le nom de la mesure.

In [ ]:
import matplotlib.pyplot as plt
import random

nb_pieces_max = 40
nb_instances = 100

list_value_LPT = []
list_mystery = []
for i in range(5,nb_pieces_max,5):
  mystery = 0
  value_LPT = 0
  for _ in range(nb_instances):

    # Définition des pièces et machines
    pieces = [random.randint(1,8)*10 for _ in range(i)]
    machines = [random.uniform(1, 3) for _ in range(3)]  # Puissances des machines (usinage plus rapide si valeur plus grande)

    #Calcul de la valeur de LPT et de la valeur mystère
    Cmax,_ = """A COMPLETER"""
    mystery += pulp.value(Cmax)
    machine_loads,_ = """A COMPLETER"""
    value_LPT += max(machine_loads)

  list_value_LPT.append("""A COMPLETER""")
  list_mystery.append("""A COMPLETER""")

#Tracé de la courbe
x= list(range(5,nb_pieces_max,5))
plt.figure()
plt.plot(x,list_value_LPT,label="LPT")
plt.plot(x,list_mystery,label="Mystère")
plt.legend()
plt.xlabel("Nombre de pièces")
plt.ylabel("Valeur de la solution")
plt.show()
